In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [ ]:
#Importation de la base détaillant les origines de la pollution
data_pollution = pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert", encoding = 'utf-8')

In [ ]:
#Dictionnaire associant les numéros de département à ceux des régions

correspondance_reg = {
    84: ['01', '03', '07', '15', '26', '38', '42', '43', '63', '69', '73', '74'],  # Auvergne-Rhône-Alpes
    27: ['21', '25', '39', '58', '70', '71', '89', '90'],  # Bourgogne-Franche-Comté
    53: ['22', '29', '35', '56'],  # Bretagne
    28: ['18', '28', '36', '37', '41', '45'],  # Centre-Val de Loire
    94: ['2A', '2B'],  # Corse
    44: ['08', '10', '51', '52', '54', '55', '57', '67', '68', '88'],  # Grand Est
    32: ['02', '59', '60', '62', '80'],  # Hauts-de-France
    11: ['75', '77', '78', '91', '92', '93', '94', '95'],  # Île-de-France
    76: ['14', '27', '50', '61', '76'],  # Normandie
    75: ['16', '17', '19', '23', '24', '33', '40', '47', '64', '79', '86', '87'],  # Nouvelle-Aquitaine
    24: ['09', '11', '12', '30', '31', '32', '34', '46', '48', '65', '66', '81', '82'],  # Occitanie
    52: ['44', '49', '53', '72', '85'],  # Pays de la Loire
    93: ['04', '05', '06', '13', '83', '84']  # Provence-Alpes-Côte d'Azur
}

correspondance_reg = {dep: region for region, deps in correspondance_reg.items() for dep in deps}

In [ ]:
#Création de la variable département à partir du code commune de l'INSEE
data_pollution['DEP'] = data_pollution['INSEE commune'].str[:2]

#Chose à laquelle il faut faire attention : 
#DEP est de type serie car il ne contient pas que de variables numériques (2A et 2B)

In [ ]:
#Création de la variable région à partir de celle du département
size = len(data_pollution['DEP'])
data_pollution['REG'] = pd.Series([0]*size)

for i in range(size) :
    data_pollution.loc[i, 'REG'] = correspondance_reg.get(data_pollution['DEP'][i],"inconnu")

In [ ]:
data_pollution

In [ ]:
REG_DEP_na = data_pollution[(data_pollution['DEP'].isna()) | (data_pollution['REG'].isna())]
REG_DEP_na.shape
#L'avantage par rapport au code précédent est qu'il n'y a plus de données perdues 
#ce qui n'était pas le cas avec le merge avec base_geo_pour_merge 

In [ ]:
len(data_pollution['REG'].unique())

In [ ]:
#Somme des niveaux de pollution par département
pollution_group_dep = data_pollution.drop(['Commune','INSEE commune', 'REG'], axis = 1).groupby('DEP', as_index=False).sum()
pollution_group_dep

In [ ]:
#Somme des niveaux de pollution par région
pollution_group_reg = data_pollution.drop(['Commune','INSEE commune', 'DEP'], axis = 1).groupby('REG', as_index = False).sum()
pollution_group_reg

In [ ]:
#Dictionnaire de correspondance des numéro de région avec leur nom
#Utile pour l'affiche des graphiques
nom_reg = {
    84: 'Auvergne-Rhône-Alpes',
    27: 'Bourgogne-Franche-Comté',
    53: 'Bretagne',
    28: 'Centre-Val de Loire',
    94: 'Corse',
    44: 'Grand Est',
    32: 'Hauts-de-France',
    11: 'Île-de-France',
    76: 'Normandie',
    75: 'Nouvelle-Aquitaine',
    24: 'Occitanie',
    52: 'Pays de la Loire',
    93: "Provence-Alpes-Côte d'Azur"
}

In [ ]:
# Conversion de la colonne 'REG' en type int
pollution_group_reg['REG'] = pollution_group_reg['REG'].astype(int)

# Création de la variable 'Nom de la région' à partir de celle de la région
pollution_group_reg['Nom de la région'] = pollution_group_reg['REG'].map(nom_reg).fillna("inconnu")

# Réaranger l'ordre des colonnes
colonnes_ordre = ['REG', 'Nom de la région'] + [colonne for colonne in pollution_group_reg.columns if colonne not in ['REG', 'Nom de la région']]
pollution_group_reg = pollution_group_reg[colonnes_ordre]

In [ ]:
pollution_group_reg

In [ ]:
set_dep_littoral = (59, 62, 80, 76, 27, 14, 50, 35, 
                      22, 29, 56, 44, 85, 17, 33, 40, 
                    64, 66, 11, 34,
                    30, 13, 83, 0o6, '2A', '2B')
print(set_dep_littoral)

In [ ]:
# Importation des bases en local pour faire tourner le code main
data_pollution.to_csv('base_pollution.csv', sep=',', index=False, encoding='utf-8')
pollution_group_reg.to_csv('pollution_group_reg.csv', sep=',', index=False, encoding='utf-8')
pollution_group_dep.to_csv('pollution_group_dep.csv', sep=',', index=False, encoding='utf-8')